In [2]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta
from connection_keys import connect_bq_client
import statsmodels.api as sm
import os
import warnings
import itertools
warnings.filterwarnings("ignore")

In [12]:
client = connect_bq_client()

sql = """
    SELECT *
    FROM `db_market_product.gpu`
    ORDER BY dt_carga asc
"""

df = client.query(sql).to_dataframe()

In [14]:
df['data'] = df.dt_carga.dt.date
df.head()

,ds_marca,ds_nome_produto,vl_preco_atual,vl_preco_com_desconto,vl_porcentagem_do_desconto,qt_quantidade_estoque,oferta,ds_disponibilidade,loja,dt_carga,data
0,Nvidia,Nvlink 3 Slot Rtx Geforce,1322.0,1322.0,0.0,0,None,False,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29
1,Gigabyte,"Gigabyte GTX 1050 Ti, 4GB, DDR5",1999.0,1999.0,0.0,0,None,False,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29
2,Nvidia,Vga Pci-e 1gb Galax Geforce Gt710 - 1gb,339.0,339.0,0.0,0,None,False,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29
3,Colorful,"Placa de Video Colorful GT 710, 1GB DDR3",349.0,349.0,0.0,93,None,True,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29
4,Asus,Placa de Vídeo Asus GeForce RTX 3080 10GB,19999.0,19999.0,0.0,0,None,False,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378682 entries, 0 to 378681
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   ds_marca                    378682 non-null  object             
 1   ds_nome_produto             378682 non-null  object             
 2   vl_preco_atual              378682 non-null  float64            
 3   vl_preco_com_desconto       378682 non-null  float64            
 4   vl_porcentagem_do_desconto  378682 non-null  float64            
 5   qt_quantidade_estoque       140745 non-null  Int64              
 6   oferta                      378682 non-null  object             
 7   ds_disponibilidade          378682 non-null  object             
 8   loja                        378682 non-null  object             
 9   dt_carga                    378682 non-null  datetime64[ns, UTC]
 10  data                        378682 non-null 

In [6]:
df.groupby(['dt_carga'])['loja'].count()
df.groupby(['data', 'loja'])['dt_carga'].nunique()
# df['data'].value_counts().sort_index().reset_index()
# df['data'] = df.dt_carga.dt.date

data        loja  
2022-09-29  Kabum     2
            Pichau    2
2022-09-30  Kabum     2
            Pichau    2
2022-10-01  Kabum     2
                     ..
2022-12-03  Pichau    2
2022-12-04  Kabum     2
            Pichau    2
2022-12-05  Kabum     1
            Pichau    1
Name: dt_carga, Length: 136, dtype: int64

In [15]:
# df_datas_loja = df.groupby(['data', 'loja'])['dt_carga'].nunique()
# df_datas_loja
maxDf = df.groupby(['data'])['dt_carga'].max()
minDf = df.groupby(['data'])['dt_carga'].min()
maxDates = maxDf.tolist()
df = df.loc[df['dt_carga'].isin(maxDates)]

# Transpostar para outra lugar depois

In [11]:
# Limpa o nome de arquivos
import os
import re
folder = './teste/'
for filename in os.listdir('./teste'):
    nome = re.sub(r'\([^)]*\)', '', filename)
    nome = re.sub(r'\s+(?=[.,?!])', '', nome)
    os.rename(os.path.join(folder, filename), os.path.join(folder, nome))